In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import gammapy


import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.convolution import Tophat2DKernel
from regions import CircleSkyRegion, RectangleSkyRegion

from gammapy.detect import compute_lima_on_off_image
from gammapy.data import DataStore
from gammapy.irf import make_mean_psf
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.cube import (
    MapDatasetMaker,
    PSFKernel,
    MapDataset,
    RingBackgroundMaker,
    SafeMaskMaker,
)

from astropy.coordinates import Angle

from gammapy.modeling.models import (
    SkyModel,
    BackgroundModel,
    PowerLawSpectralModel,
    PowerLaw2SpectralModel,
    PointSpatialModel,
    ExpCutoffPowerLawSpectralModel,
    SkyDiffuseCube,
    TemplateSpatialModel
)
from gammapy.modeling import Fit
from astropy.time import Time

src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")


from gammapy.cube import (
    SafeMaskMaker,
    PSFKernel,
    MapDataset,
    MapDatasetMaker,
    MapDatasetOnOff,
    RingBackgroundMaker,
)


In [7]:
emin, emax = [0.5, 100] * u.TeV

energy_axis = MapAxis.from_bounds(
    emin.value, emax.value, 20, unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    coordsys="GAL",
    proj="CAR",
    axes=[energy_axis],
)


In [8]:
# Define which data to use and print some information
data_store = DataStore.from_dir("$GAMMAPY_DATA/ash_stereo_Prod17_Calib0834_thsq64")
data_store.info()

Data store:
HDU index table:
BASE_DIR: /home/samuel/code/gammapy_data/ash_stereo_Prod17_Calib0834_thsq64
Rows: 122853
OBS_ID: 18092 -- 151486
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_2d', 'edisp_2d', 'events', 'gti', 'psf_3gauss', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 20485



In [9]:
selection = dict(type='sky_circle', frame='galactic',
                 lon=Angle(0, 'deg'),
                 lat=Angle(0, 'deg'),
                 radius=Angle(2, 'deg'),
                 border=Angle(0, 'deg'))

obs_table = data_store.obs_table.select_observations(selection)

In [10]:
t2004  = dict(type='time_box', time_range= Time(['2004-01-01T00:00:00', '2004-12-31T23:59:59']))
t2005  = dict(type='time_box', time_range= Time(['2005-01-01T00:00:00', '2005-12-31T23:59:59']))
t2006  = dict(type='time_box', time_range= Time(['2006-01-01T00:00:00', '2006-12-31T23:59:59']))
t2007  = dict(type='time_box', time_range= Time(['2007-01-01T00:00:00', '2007-12-31T23:59:59']))
t2008  = dict(type='time_box', time_range= Time(['2008-01-01T00:00:00', '2008-12-31T23:59:59']))
t2009  = dict(type='time_box', time_range= Time(['2009-01-01T00:00:00', '2009-12-31T23:59:59']))
t2010  = dict(type='time_box', time_range= Time(['2010-01-01T00:00:00', '2010-12-31T23:59:59']))
t2011  = dict(type='time_box', time_range= Time(['2011-01-01T00:00:00', '2011-12-31T23:59:59']))
t2012  = dict(type='time_box', time_range= Time(['2012-01-01T00:00:00', '2012-12-31T23:59:59']))
t2013  = dict(type='time_box', time_range= Time(['2013-01-01T00:00:00', '2013-12-31T23:59:59']))
t2014  = dict(type='time_box', time_range= Time(['2014-01-01T00:00:00', '2014-12-31T23:59:59']))
t2015  = dict(type='time_box', time_range= Time(['2015-01-01T00:00:00', '2015-12-31T23:59:59']))
t2016  = dict(type='time_box', time_range= Time(['2016-01-01T00:00:00', '2016-12-31T23:59:59']))
t2017  = dict(type='time_box', time_range= Time(['2017-01-01T00:00:00', '2017-12-31T23:59:59']))
t2018  = dict(type='time_box', time_range= Time(['2018-01-01T00:00:00', '2018-12-31T23:59:59']))
t2019  = dict(type='time_box', time_range= Time(['2019-01-01T00:00:00', '2019-12-31T23:59:59']))

In [11]:
year_intervals = { 2004 : t2004, 2005 : t2005, 2006 : t2006, 2007 : t2007,
                      2008 : t2008, 2009 : t2009, 2010 : t2010, 2011 : t2011,
                      2012 : t2012, 2013 : t2013, 2014 : t2014, 2015 : t2015,
                      2016 : t2016, 2017 : t2017, 2018 : t2018, 2019 : t2019}

yearly_obs = dict()

for year in range(2004,2020) :
    
    obs_table_year = obs_table.select_observations(year_intervals[year])
    ids = obs_table_year["OBS_ID"].tolist()
    observations_year = data_store.get_observations(ids, skip_missing=True)
    
    for obs in observations_year:
        try:
            obs.aeff
            obs.edisp
            obs.psf
        except:
            ids.remove(obs.obs_id)
            print("Observation retirée : " + str(obs.obs_id))
            
    observations_year = data_store.get_observations(ids, skip_missing=True)
    yearly_obs[year] = observations_year

Found multiple HDU matching: OBS_ID = 20191, HDU_TYPE = psf, HDU_CLASS = None. Returning the first entry, which has HDU_TYPE = psf and HDU_CLASS = psf_3gauss
Found multiple HDU matching: OBS_ID = 20193, HDU_TYPE = psf, HDU_CLASS = None. Returning the first entry, which has HDU_TYPE = psf and HDU_CLASS = psf_3gauss
Found multiple HDU matching: OBS_ID = 20194, HDU_TYPE = psf, HDU_CLASS = None. Returning the first entry, which has HDU_TYPE = psf and HDU_CLASS = psf_3gauss


Observation retirée : 20191
Observation retirée : 20193
Observation retirée : 20194
Observation retirée : 31539
Observation retirée : 31577
Observation retirée : 31578
Observation retirée : 31579
Observation retirée : 31580


In [12]:
%%time

stacked_datasets = dict()

for year in range(2004,2020):
    
    stacked = MapDataset.create(geom)
    datasets = []
    
    maker = MapDatasetMaker(selection=["counts", "exposure", "background"])
    maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=2.0 * u.deg)

    for obs in yearly_obs[year]:
        cutout = stacked.cutout(obs.pointing_radec, width="5 deg")
        dataset = maker.run(cutout, obs)
        dataset = maker_safe_mask.run(dataset, obs)
        datasets.append(dataset)
        
        stacked.stack(dataset)
    
    stacked_datasets[year] = stacked

CPU times: user 19min 54s, sys: 13.6 s, total: 20min 8s
Wall time: 20min 15s


In [13]:
spectrum = PowerLaw2SpectralModel(index=2.3)

maps2D = dict()

for year in range(2004,2020):
    
    dataset_2d = stacked_datasets[year].to_image(spectrum=spectrum)

    maps2D[year] = {
        "counts": dataset_2d.counts,
        "exposure": dataset_2d.exposure,
        "background": dataset_2d.background_model.map,
    }

In [ ]:
# écriture des maps 2D



In [15]:
src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

psf_kernels = dict()

for year in range(2004,2020):
    # mean PSF

    geom2d = maps2D[year]["exposure"].geom
    
    
    table_psf = make_mean_psf(yearly_obs[year], src_pos)

    table_psf_2d = table_psf.table_psf_in_energy_band(
        (emin, emax), spectrum=spectrum
    )

    # PSF kernel used for the model convolution
    psf_kernel = PSFKernel.from_table_psf(
        table_psf_2d, geom2d, max_radius="0.3 deg"
    )
    psf_kernels[year] = psf_kernel


In [17]:
# define a mask 

fit_region = RectangleSkyRegion(src_pos, 3*u.deg, 1*u.deg)

g0901_region = CircleSkyRegion(SkyCoord(0.8791,  +0.0893, unit="deg", frame="galactic"), 0.1*u.deg)

bizarre_region = CircleSkyRegion(SkyCoord(358.6,  -0.6, unit="deg", frame="galactic"), 0.4 * u.deg)

fit_mask = geom.region_mask([fit_region])*geom.region_mask([g0901_region ,bizarre_region ] , inside=False)

fit_mask = Map.from_geom(geom, data=fit_mask)

In [18]:
# Modèle de la source

src_pos = SkyCoord(359.94, -0.04, unit="deg", frame="galactic")

spatial_model = PointSpatialModel(
            lon_0 = src_pos.l, 
            lat_0 = src_pos.b, 
            frame = "galactic")

spectral_model = PowerLawSpectralModel(
            #emin = 0.5* u.TeV, 
            #emax = 100* u.TeV, 
            reference = 1.0 * u.TeV,
            index = 2.3, 
            amplitude = "1.5e-12 cm-2 s-1 TeV-1")

model = SkyModel(
            spatial_model = spatial_model, 
            spectral_model = spectral_model, 
            name = "SgrA*")

model.parameters["index"].frozen = True
model.parameters["lon_0"].frozen = True
model.parameters["lat_0"].frozen = True

In [27]:
# Modèle de l'émission diffuse

CSmap = Map.read("$GAMMAPY_DATA/CS_reproj.fits") # à moduler par une gaussienne en longitude

coords = CSmap.geom.get_coord()
lon = coords.skycoord.l.wrap_at(180*u.deg).to_value('deg')
factor = np.exp(-0.5*(lon)**2/1.0**2)

# seuiller les valeurs pour enlever les valeurs négatives liées au bruit

CSmap.data*=factor

CSmap.data = CSmap.data.clip(10.0)

total_CS = CSmap.quantity*CSmap.geom.solid_angle()

CSmap.data /= total_CS.sum().value
diffuse_gal = TemplateSpatialModel(CSmap)

diffuse_gal.parameters["norm"].value = 1.0
diffuse_gal = TemplateSpatialModel(CSmap)

powerlaw2 =  PowerLawSpectralModel(
            #emin =0.5* u.TeV, 
            #emax = 100* u.TeV, 
            reference = 1.0 * u.TeV,
            index = 2.3, 
            amplitude = "1e-10 cm-2 s-1 TeV-1")


model_diff = SkyModel(
            spatial_model = diffuse_gal, 
            spectral_model = powerlaw2, 
            name = "DE")

model_diff.parameters["index"].frozen = True
model_diff.parameters["norm"].frozen = True

Invalid value BUNIT='Count' in FITS header. Setting empty unit.


In [31]:
# modèle de fond (le faire année par année)
bckgrd_models = dict()

for year in range(2004,2020):
    bckgrd_model = BackgroundModel(maps2D[year]["background"])
    background_model.parameters["norm"].frozen = True
    background_model.parameters["tilt"].frozen = True
    
    bckgrd_models[year] = bckgrd_model

In [34]:
datasets2D = dict()

for year in range(2004,2020):
    dataset = MapDataset(
        models=model.copy() + model_diff.copy(),
        counts=maps2D[year]["counts"],
        exposure=maps2D[year]["exposure"],
        background_model=background_model,
        mask_fit=fit_mask,
        psf=psf_kernels[year],
        )
    datasets2D[year] = dataset

In [33]:
%%time

results = dict()

for year in range(2004,2020):
    fit = Fit([datasets2D[year]])
    result = fit.run()
    
    results[year] = result

RuntimeError: exception was raised in user function
User function arguments:
    par_000_amplitude = +1.500000
    par_001_amplitude = +1.000000
Original python exception in user function:
IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 20
  File "/home/samuel/anaconda3/envs/gammapy-0.15/lib/python3.7/site-packages/gammapy/modeling/iminuit.py", line 17, in fcn
    return self.function()
  File "/home/samuel/anaconda3/envs/gammapy-0.15/lib/python3.7/site-packages/gammapy/modeling/datasets.py", line 129, in stat_sum
    stat_sum += dataset.stat_sum()
  File "/home/samuel/anaconda3/envs/gammapy-0.15/lib/python3.7/site-packages/gammapy/cube/fit.py", line 605, in stat_sum
    return cash_sum_cython(counts[self.mask.data], npred[self.mask.data])


In [ ]:
print(model)

In [ ]:
result.parameters.to_table()

In [ ]:
result.parameters.correlation[:4, :4]